In [15]:
from opensky_api import OpenSkyApi, FlightTrack
import os.path
import pandas as pd
import folium
import time
import os 
from dotenv import load_dotenv
from folium.plugins import MarkerCluster
load_dotenv()
import avwx
opensky_user = os.getenv('OPENSKY_USER')
opensky_api_key = os.getenv('OPENSKY_API_KEY')

# Initialize the API (can also be authenticated) 
# dont be dumb if you have the keys
api = OpenSkyApi(opensky_user, opensky_api_key)

# Fetch Live Flights in the Air or on the Ground in the United States

In [16]:
# fetch flights only in the united states 
states = api.get_states()
data = []
for s in states.states:
    data.append({
            "ICAO24": s.icao24,
            "Callsign": s.callsign,
            "Origin Country": s.origin_country,
            "Latitude": s.latitude,
            "Longitude": s.longitude,
            "Baro Altitude": s.baro_altitude,
            "Geo Altitude": s.geo_altitude,
            "Velocity": s.velocity,
            "True Track": s.true_track,
            "Vertical Rate": s.vertical_rate,
            "On Ground": s.on_ground,
            "Category": s.category,
            "Last Contact": s.last_contact,
            "Position Source": s.position_source,
            "Sensors": s.sensors,
            "SPI": s.spi,
            "Squawk": s.squawk,
            "Time Position": s.time_position
        })

us_flights_data = [flight for flight in data if flight["Origin Country"] == "United States"]
us_flights_df = pd.DataFrame(us_flights_data)
us_flights_df

,ICAO24,Callsign,Origin Country,Latitude,Longitude,Baro Altitude,Geo Altitude,Velocity,True Track,Vertical Rate,On Ground,Category,Last Contact,Position Source,Sensors,SPI,Squawk,Time Position
0,a665ae,IFL511,United States,36.8688,-87.3000,1645.92,1684.02,140.12,123.16,2.28,False,0,1709182580,0,None,False,None,1.709183e+09
1,ab1644,UAL1669,United States,41.5497,-93.6803,220.98,327.66,62.98,132.68,-3.25,False,4,1709182347,0,None,False,1416,1.709182e+09
2,ac1c16,DOW88,United States,42.6794,-98.3585,12481.56,12367.26,253.22,49.45,0.33,False,0,1709182580,0,None,False,1374,1.709183e+09
3,aa56da,UAL1821,United States,43.2902,-97.3515,11894.82,11757.66,241.14,56.75,0.00,False,4,1709182581,0,None,False,1466,1.709183e+09
4,a5f84e,N484JS,United States,29.1321,-98.5430,1866.90,1950.72,83.88,344.34,0.00,False,0,1709182581,0,None,False,None,1.709183e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102,a0c7b3,GTI3782,United States,41.1628,-100.1193,10363.20,10355.58,213.89,260.45,0.00,False,0,1709182581,0,None,False,3661,1.709183e+09
2103,a6d83c,DAL518,United States,40.4247,-74.1370,10668.00,10637.52,305.90,50.53,0.00,False,0,1709182581,0,None,False,5734,1.709183e+09
2104,a05dba,UAL1787,United States,41.3783,-96.0651,807.72,861.06,125.51,78.41,-1.63,False,1,1709182581,0,None,False,0023,1.709183e+09
2105,a9607d,JBU2071,United States,25.7590,-79.1455,6454.14,6842.76,240.37,119.91,8.45,False,4,1709182357,0,None,False,None,1.709182e+09


# Load US airport data

In [28]:
# Load the airport data from the CSV file
airport_df = pd.read_csv('airports.csv')
# US only airports
us_airports_df = airport_df[airport_df['iso_country'] == 'US']
# now only get medium airport and large airports
us_airports_df = us_airports_df[us_airports_df['type'].isin(['small_airport', 'medium_airport', 'large_airport'])]
print(us_airports_df.columns)

In [31]:
# m = folium.Map(location=[37, -102])
# marker_cluster = MarkerCluster().add_to(m)

# for i, row in us_airports_df.iterrows():
#     folium.Marker([row['Latitude'], row['Longitude']], popup=row['Callsign']).add_to(marker_cluster)

# m.save('us_airports.html')

# Weather: AVWX library

In [5]:
import avwx
from avwx.forecast import *
import asyncio

# Function to Fetch Data
async def fetch_airport_data_async(airport_codes):
    airport_data = {}
    for airport_code in airport_codes:
        data = {}
        # Fetch METAR data
        try:
            metar = avwx.Metar(airport_code)
            if await metar.async_update():
                data["metar"] = {
                    "raw": metar.raw,
                    "flight_rules": metar.data.flight_rules,
                    "summary": metar.summary, 
                }
            else:
                data["metar"] = None
        except Exception as e:
            print(f"Error fetching METAR data for {airport_code}: {e}")
            data["metar"] = None

        # Fetch TAF data
        try:
            taf = avwx.Taf(airport_code)
            if await taf.async_update():
                data["taf"] = {
                    "raw": taf.translations,
                    "forecast": [
                        {
                            "flight_rules": line.flight_rules,
                            "start_time": line.start_time.dt.strftime('%d-%H:%M'),
                            "end_time": line.end_time.dt.strftime('%d-%H:%M'),
                        }
                        for line in taf.data.forecast
                    ],
                }
            else:
                data["taf"] = None
        except Exception as e:
            print(f"Error fetching TAF data for {airport_code}: {e}")
            data["taf"] = None
        
        airport_data[airport_code] = data
    return airport_data


async def main():
    airports = us_airports_df['gps_code'].tolist()
    data = await fetch_airport_data_async(airports)
    # html_output = generate_summary_html_safe(data)
#     # with open("weather_summary.html", "w") as file:
#     #     file.write(html_output)
    return data  # Return the data instead of printing it

# Run the Main Function
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    data = loop.run_until_complete(main()) 

Error fetching METAR data for 00AA: Station doesn't start with a recognized character set
Error fetching TAF data for 00AA: Station doesn't start with a recognized character set
Error fetching METAR data for 00AK: Station doesn't start with a recognized character set
Error fetching TAF data for 00AK: Station doesn't start with a recognized character set
Error fetching METAR data for 00AL: Station doesn't start with a recognized character set
Error fetching TAF data for 00AL: Station doesn't start with a recognized character set
Error fetching METAR data for 00AS: Station doesn't start with a recognized character set
Error fetching TAF data for 00AS: Station doesn't start with a recognized character set
Error fetching METAR data for 00AZ: Station doesn't start with a recognized character set
Error fetching TAF data for 00AZ: Station doesn't start with a recognized character set
Error fetching METAR data for 00CA: Station doesn't start with a recognized character set
Error fetching TAF d

## Find Icing Conditions

In [8]:
# Initialize a list to hold airports with icing conditions
airports_with_icing = []

# Iterate through the dictionary
for airport_code, weather_info in data.items():
    if 'taf' in weather_info and weather_info['taf'] is not None:  # Ensure TAF data is available
        for forecast_line in weather_info['taf']['raw'].forecast:  # Loop through each forecast line
            if forecast_line.icing:  # Check if icing is mentioned
                # Since icing is an empty string in your example, you'd check its presence differently
                # This is a placeholder condition, adjust according to how icing data is represented
                if forecast_line.icing != '':  # Adjust this condition based on actual data representation
                    airports_with_icing.append(airport_code)
                    break  # Stop checking further lines for this airport

# Now, 'airports_with_icing' contains the codes of airports with icing conditions
print("Airports with icing conditions:", airports_with_icing)

Airports with icing conditions: ['KBAB', 'KCBM', 'KCEF', 'KCVS', 'KDMA', 'KGTB', 'KHMN', 'KLTS', 'KMTC', 'KMUO', 'KNFW', 'KNHK', 'KNKT', 'KNLC', 'KNUW', 'KRDR', 'KSKA', 'KSZL', 'KTCM', 'KWRB']


In [35]:
us_airports_df[['latitude', 'longitude']] = us_airports_df['coordinates'].str.split(',', expand=True)
# now matvh the airportts with icing conditions to the us_airports_df
# now get the coordinates from the usa_airports_df and plot them on the map
m = folium.Map(location=[37, -102])
marker_cluster = MarkerCluster().add_to(m)

for i, row in us_airports_df.iterrows():
    if row['gps_code'] in airports_with_icing:
        folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(marker_cluster)
        
m.save('us_airports_icing.html')
m

## Generates HTML for airport TAFs



In [ ]:
# # Function to Generate HTML
# def generate_summary_html_safe(airports_data):
#     # Styles
#     styles = """
#     <style>
#         table {
#             border-collapse: collapse;
#             width: 80%;
#             margin: 20px auto;
#         }
#         th, td {
#             text-align: left;
#             padding: 8px 12px;
#             border: 1px solid #ddd;
#         }
#         tr:nth-child(even) {
#             background-color: #f4f4f4;
#         }
#         th {
#             background-color: #f2f2f2;
#         }
#         .airport-name {
#             font-weight: bold;
#             font-size: 1.5em;
#             text-align: center;
#             margin-top: 20px;
#         }
#         .section-header {
#             font-weight: bold;
#             margin-top: 40px;
#             font-size: 1.3em;
#             text-align: center;
#         }
#     </style>
#     """
    
#     # Rows
#     metar_rows_html = ""
#     taf_rows_html = ""
#     for airport, data in airports_data.items():
#         # METAR
#         if data["metar"]:
#             metar = data["metar"]["summary"].split(", ")
#             metar_rows_html += f"<tr>"
#             metar_rows_html += f"<td>{airport}</td>"
#             metar_rows_html += f"<td>{data['metar']['flight_rules']}</td>"
#             for item in metar:
#                 metar_rows_html += f"<td>{item}</td>"
#             metar_rows_html += "</tr>"
        
#         # TAF
#         if data["taf"]:
#             taf_rows_html += f"<tr>"
#             taf_rows_html += f"<td>{airport}</td>"
#             taf_forecast = ', '.join([f"{forecast['flight_rules']} from {forecast['start_time']} to {forecast['end_time']}" for forecast in data['taf']['forecast']])
#             taf_rows_html += f"<td>{taf_forecast}</td>"
#             taf_rows_html += "</tr>"

#     # Complete HTML Structure
#     html = f"""
#     <html>
#         <head>
#             <title>Weather Summary</title>
#             {styles}
#         </head>
#         <body>
#             <div class='airport-name'>Weather Summary for {', '.join(airports_data.keys())}</div>
#             <div class='section-header'>METAR</div>
#             <table>
#                 <tr><th>Airport</th><th>Flight Rules</th><th>Wind</th><th>Visibility</th><th>Temperature</th><th>Dew Point</th><th>Pressure</th><th>Clouds</th><th>Conditions</th></tr>
#                 {metar_rows_html}
#             </table>
#             <div class='section-header'>TAF</div>
#             <table>
#                 <tr><th>Airport</th><th>Forecast</th></tr>
#                 {taf_rows_html}
#             </table>
#         </body>
#     </html>
#     """
#     return html

# Async Main Function to Drive the Process
# Async Main Function to Drive the Process

In [ ]:
import nest_asyncio
nest_asyncio.apply()

taf = avwx.Taf('KJFK')

# Use asyncio.run() to run taf.update()
asyncio.run(taf.update())
print(taf.raw)




In [ ]:



taf = avwx.Taf('KJFK')

# Use asyncio.run() to run taf.update()
asyncio.run(taf.update())
print(taf.raw)